In [1]:
import pygmo as pg  # See https://esa.github.io/pygmo2/install.html for installation (conda instructions)

import numpy as np
import pandas as pd
from time import time

from airsenal.framework.utils import NEXT_GAMEWEEK, CURRENT_SEASON, list_players, get_latest_prediction_tag
from airsenal.framework.team import Team

In [2]:
gw_start = NEXT_GAMEWEEK
num_gw = 3
gw_range = list(range(gw_start, min(38, num_gw)))

tag = get_latest_prediction_tag()

#positions = ["FWD", "MID", "DEF", "GK"]
#pts = {}
#for pos in positions:
#    pts[pos] = get_predicted_points(gw_range, tag, position=pos)
    
#pts = get_predicted_points(gw_range, tag)

In [3]:
# PyGMO User Defined Problem
class OptTeam:
    def __init__(self, gw_range, tag, budget=100, season=CURRENT_SEASON, bench_boost_gw=None, triple_captain_gw=None):
        self.gw_range = gw_range
        self.start_gw = min(gw_range)
        self.tag = tag
        self.n_players = 15
        self.budget = budget
        self.season = season
        players = list_players(season=season, gameweek=self.start_gw)
        self.min_id = min([p.player_id for p in players])
        self.max_id = max([p.player_id for p in players])
        self.bench_boost_gw = bench_boost_gw
        self.triple_captain_gw = triple_captain_gw

          
    def fitness(self, player_ids):
        """
        PyGMO required function.
        The objective function to minimise. And constraints to evaluate.
        """
        # Make team from player IDs
        team = Team()
        for idx in player_ids:
            team.add_player(int(idx), season=self.season, gameweek=self.start_gw)
            
        # Check team is valid
        if not team.is_complete():
            return [0]
        
        # Calc expected points
        score = 0.0
        for gw in self.gw_range:
            if gw == self.bench_boost_gw:
                score += team.get_expected_points(gw, self.tag, bench_boost=True)
            elif gw == self.triple_captain_gw:
                score += team.get_expected_points(gw, self.tag, triple_captain=True)
            else:
                score += team.get_expected_points(gw, self.tag)
        
        return [-score]
    
    def get_bounds(self):
        """
        PyGMO required function.
        Defines min and max value for each parameter.
        """
        return ([self.min_id]*self.n_players, [self.max_id]*self.n_players)
    
    def get_nec(self):
        """PyGMO function.
        Defines number of equality constraints."""
        return 0
    
    def get_nix(self):
        """
        PyGMO function.
        Number of integer dimensions.
        """
        return self.n_players
    
    def gradient(self, x):
        return pg.estimate_gradient_h(lambda x: self.fitness(x), x)
    


In [4]:
# Build problem
opt_team = OptTeam(
    gw_range,
    tag,
)

prob = pg.problem(opt_team)

print(prob)

Problem name: <class '__main__.OptTeam'>
	Global dimension:			15
	Integer dimension:			15
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [1, 1, 1, 1, 1, ... ]
	Upper bounds: [483, 483, 483, 483, 483, ... ]
	Has batch fitness evaluation: false

	Has gradient: true
	User implemented gradient sparsity: false
	Expected gradients: 15
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0
	Gradient evaluations: 0

	Thread safety: none



In [5]:
# Create algorithm to solve problem with
#algo = pg.algorithm(uda = pg.sga(gen=3))
algo = pg.algorithm(uda = pg.gaco(gen=100))
#algo = pg.algorithm(uda = pg.ihs(gen=1000))
algo.set_verbosity(1)
print(algo)

# population of problems
pop = pg.population(prob=prob, size=1000)


Algorithm name: GACO: Ant Colony Optimization [stochastic]
	Thread safety: basic

Extra info:
	Generations: 100
	Accuracy parameter: 0.01
	Improvement stopping criterion: 100000
	Evaluation stopping criterion: 100000
	Focus parameter: 0
	Kernel: 63
	Oracle parameter: 0
	Pseudo-random number generator (Marsenne Twister 19937): 2942857457 3008076512 3336899884 1018518158 1659156490 386367324 1688304466 2180649670 1326467932 894271674 2123191148 3546431244 116058103 4237407104 3407998397 4107237893 2085771886 1810633948 3109650499 4089012920 4111927003 1420249933 207642898 3044470321 1154106679 256980071 1659227069 3772054855 1282232304 2189394738 2412999182 348678875 3292420999 4230987573 245651056 1445464659 1267725438 488402240 2943244902 3987025307 2260292896 26986643 2161233193 3653170466 3110887217 1159256652 62674319 3992995994 2502264205 3878697116 2762763757 342181502 1411399146 1195913452 337978039 4181434730 4089896869 3232764599 2993009982 348232935 2343274079 3458123758 10505

In [6]:
# solve problem
pop = algo.evolve(pop)

print("Best score:", -pop.champion_f[0], "pts")

Best score: 70.95075274817736 pts


In [7]:
team = Team()
for idx in pop.champion_x:
    team.add_player(int(idx), season=opt_team.season, gameweek=opt_team.start_gw)

pts = team.get_expected_points(opt_team.start_gw, tag)
print(f"GW{opt_team.start_gw}: {pts:.0f} pts")
print(team)

GW1: 37 pts

=== starting 11 ===


== GK ==

Martin Dubravka (NEW)

== DEF ==

Tyrick Mitchell (CRY)
Toby Alderweireld (TOT)
Trent Alexander-Arnold (LIV)(C)
Luke Thomas (LEI)(VC)
George Baldock (SHU)

== MID ==

James Maddison (LEI)
John Fleck (SHU)
Matteo Guendouzi (ARS)

== FWD ==

Andy Carroll (NEW)
Lys Mousset (SHU)

=== subs ===

Anwar El Ghazi (AVL)
Odion Ighalo (MUN)
Harry Wilson (LIV)
Illan Meslier (LEE)

